In [ ]:
!pip install streamlit
!python -m pip install --upgrade streamlit-extras
!pip install PyPDF2
!pip install chromadb
!pip install langchain
!pip install langchain_community
!pip install langchain-groq
!pip install groq
!pip install llama-index-llms-groq
!pip install llama-index
!pip install llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%%writefile app.py
import streamlit as st
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from llama_index.llms.groq import Groq
from langchain_groq import ChatGroq
from langchain.chains import ConversationalRetrievalChain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import ConversationBufferMemory

groq_api_key = "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"
groq_llm = ChatGroq(
    model="llama3-8b-8192",
    api_key=groq_api_key
)

with st.sidebar:
    st.title('DocChat')
    st.markdown('''
                ## About
                This is an LLM-powered chatbot application built using:
                - Streamlit
                - Chroma DB
                - PyPDF2
                - LangChain
                - Groq
                - Google Colab
    ''')

def main():
    st.header("PDF Question Answering App")
    uploaded_file = st.file_uploader("Upload a PDF file", type=["pdf"])
    if uploaded_file is not None:
        st.write(f"Processing uploaded file: {uploaded_file.name}")
        pdf = PyPDF2.PdfReader(uploaded_file)
        pdf_text = ""
        for page in pdf.pages:
            pdf_text += page.extract_text()
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
        )
        chunks = text_splitter.split_text(pdf_text)
        metadata = [{"source": f"{i}-pl"} for i in range(len(chunks))]
        embeddings = HuggingFaceEmbedding(model_name="hkunlp/instructor-base")
        docsearch = Chroma.from_texts(chunks, embeddings, metadatas=metadata)
        message_history = ChatMessageHistory()
        memory = ConversationBufferMemory(
            memory_key="chat_history",
            output_key="answer",
            chat_memory=message_history,
            return_messages=True,
        )
        chain = ConversationalRetrievalChain.from_llm(
            llm=groq_llm,
            chain_type="stuff",
            retriever=docsearch.as_retriever(),
            memory=memory,
            return_source_documents=True,
        )
        st.write("File processing complete. You can now ask your questions!")
        if "chain" not in st.session_state:
            st.session_state.chain = chain
        user_input = st.text_input("Ask a question about the PDF:")
        if user_input:
            response = st.session_state.chain(user_input)
            answer = response["answer"]
            source_documents = response["source_documents"]
            source_texts = [doc.page_content for doc in source_documents]
            st.write("Answer:")
            st.write(answer)
            st.write("Sources:")
            for source_text in source_texts:
                st.write(source_text)

if __name__ == "__main__":
    main()

Overwriting app.py


In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 35.233.151.222


In [ ]:
!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501

npx: installed 22 in 1.719s
your url is: https://bright-parts-thank.loca.lt
^C


Once you run all the cells, click on the link generated by the last cell and in the website, copy-paste the endpoint ip generated in the last second cell into that website.